In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import os
import sys
cm_folder = "/Users/simon.hughes/GitHub/NlpResearch/PythonNlpResearch/notebooks/Causal Model/"
src_path = os.path.join(cm_folder, "src")
sys.path.append(src_path)

In [3]:
from typing import Any

import dill
import numpy as np

from CrossValidation import cross_validation
from structured_perceptron import StructuredPerceptron
from Settings import Settings

from window_based_tagger_config import get_config
from crel_helper import get_cr_tags
from crel_processing import essay_to_crels_cv
from evaluation import evaluate_model_essay_level, get_micro_metrics, metrics_to_df
from feature_normalization import min_max_normalize_feats
from function_helpers import get_function_names
from results_procesor import ResultsProcessor
from train_parser import essay_to_crels, create_extractor_functions
from cost_functions import micro_f1_cost_plusepsilon
from train_reranker import train_model, train_instance, get_essays_for_data, evaluate_ranker
from searn_parser_breadth_first import SearnModelBreadthFirst
from causal_model_features import CausalModelType
from feature_extraction import get_features_from_probabilities
from results_procesor import ResultsProcessor
from filter_features import filter_feats

from wordtagginghelper import merge_dictionaries
from results_procesor import ResultsProcessor, __MICRO_F1__
from evaluation import add_cr_labels

from random import shuffle
from joblib import Parallel, delayed
from collections import defaultdict

In [4]:
def train_model_fold(xs_train, xs_test, name2essay, learning_rate, max_update_items, set_cr_tags,\
                     initial_weight, max_epochs, early_stop_iters):

    mdl = StructuredPerceptron(
        learning_rate=learning_rate, max_update_items=max_update_items, initial_weight=initial_weight)

    return train_model(mdl, xs_train=xs_train, xs_test=xs_test, name2essay=name2essay,
            max_epochs=max_epochs, early_stop_iters=early_stop_iters, set_cr_tags=set_cr_tags,
            train_instance_fn=train_instance,
            verbose=False, return_metrics=True, early_stopping=False)

def train_model_parallel(cv_folds, name2essay, learning_rate, max_update_items, set_cr_tags, \
                         initial_weight, max_epochs=5, early_stop_iters=5, n_jobs=None):

    if n_jobs == None:
        n_jobs = len(cv_folds)
    try:
        results = Parallel(n_jobs=n_jobs)(
            delayed(train_model_fold)(train, test, name2essay, learning_rate, max_update_items, set_cr_tags, \
                                      initial_weight, max_epochs, early_stop_iters)
            for (train, test) in cv_folds)

        f1s = []
        for tpl in results:
            best_test_f1, best_iterations, train_ys_bytag, train_pred_ys_bytag, test_ys_bytag, test_pred_ys_bytag, num_feats = tpl
            f1s.append(best_test_f1)
        return np.mean(f1s)
    except KeyboardInterrupt:
        print("Process stopped by user")

def train_model_parallel_logged(training_collection_name: str, results_processor: ResultsProcessor,
                                feat_extractors, params,
                                cv_folds, name2essay,
                                learning_rate: float, max_update_items:int, set_cr_tags, \
                                initial_weight: float,  max_epochs=5, early_stop_iters=5, n_jobs=None):
    if not n_jobs or n_jobs == None:
        n_jobs = len(cv_folds)

    try:
        results = Parallel(n_jobs=n_jobs)(
            delayed(train_model_fold)(train, test, name2essay, learning_rate, max_update_items, set_cr_tags, \
                                      initial_weight, max_epochs, early_stop_iters)
            for (train, test) in cv_folds)

        cv_sent_td_ys_by_tag, cv_sent_td_predictions_by_tag = defaultdict(list), defaultdict(list)
        cv_sent_vd_ys_by_tag, cv_sent_vd_predictions_by_tag = defaultdict(list), defaultdict(list)

        f1s = []
        feats = []
        for tpl in results:
            best_test_f1, best_iterations, train_ys_bytag, train_pred_ys_bytag, test_ys_bytag, test_pred_ys_bytag, num_feats = tpl
            feats.append(num_feats)
            f1s.append(best_test_f1)

            merge_dictionaries(train_ys_bytag, cv_sent_td_ys_by_tag)
            merge_dictionaries(test_ys_bytag, cv_sent_vd_ys_by_tag)

            merge_dictionaries(train_pred_ys_bytag, cv_sent_td_predictions_by_tag)
            merge_dictionaries(test_pred_ys_bytag, cv_sent_vd_predictions_by_tag)


        ALGO = "Structured Perceptron"
        validation_collection = training_collection_name.replace("_TD", "_VD")

        # extractors = list(map(lambda fn: fn.func_name, feat_extractors))
        extractors = list(feat_extractors)

        parameters = {
            "learning_rate":        learning_rate,
            "loss_type":            "None",
            "max_update_items":     max_update_items,
            "initial_weight":       initial_weight,

            "max_epochs":           max_epochs,
            "early_stopping_iters": early_stop_iters,

            "extractors":           extractors,

            # Add in number of features
            "num_feats_per_fold":   feats,
            "num_feats_MEAN":       np.mean(feats)
        }
        # add in additional parameters not passed in
        parameters.update(params)

        wd_td_objectid = results_processor.persist_results(training_collection_name,
                                                           cv_sent_td_ys_by_tag,
                                                           cv_sent_td_predictions_by_tag,
                                                           parameters, ALGO)

        wd_vd_objectid = results_processor.persist_results(validation_collection,
                                                           cv_sent_vd_ys_by_tag,
                                                           cv_sent_vd_predictions_by_tag,
                                                           parameters, ALGO)

        avg_f1 = float(results_processor.get_metric(validation_collection, wd_vd_objectid, __MICRO_F1__)["f1_score"])
        return avg_f1
    except KeyboardInterrupt:
        print("Process stopped by user")

In [5]:
# Data Set Partition
CV_FOLDS = 5
MIN_FEAT_FREQ = 5

# Global settings
settings = Settings()
CAUSAL_MODEL_TYPE = CausalModelType.SKIN_CANCER
root_folder = settings.data_directory + "SkinCancer/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"

crels_folder = "./crels/SC"
coref_root = root_folder + "CoReference/"
coref_output_folder = coref_root + "CRel/"

MONGO_COLLECTION = "SC_STR_PCPTRN_RE-RANKER_HYPER_PARAM_TD"
MONGO_TEST_COLLECTION = "TEST_SC_STR_PCPTRN_RE-RANKER_TD"

config = get_config(training_folder)
results_processor = ResultsProcessor(dbname="metrics_causal_model_reranker")

train_fname = coref_output_folder + "training_crel_anatagged_essays_most_recent_code.dill"
with open(train_fname, "rb") as f:
    pred_tagged_essays_train = dill.load(f)

test_fname = coref_output_folder + "test_crel_anatagged_essays_most_recent_code.dill"
with open(test_fname, "rb") as f:
    pred_tagged_essays_test = dill.load(f)

print(len(pred_tagged_essays_train), len(pred_tagged_essays_test))

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
870 218


In [6]:
len(pred_tagged_essays_train) + len(pred_tagged_essays_test)

1088

In [6]:
cr_tags = get_cr_tags(train_tagged_essays=pred_tagged_essays_train, tag_essays_test=pred_tagged_essays_test)

set_cr_tags = set(cr_tags)
list(set_cr_tags)[0:10]

['Causer:7->Result:1',
 'Causer:4->Result:13',
 'Causer:5->Result:50',
 'Causer:1->Result:4',
 'Causer:14->Result:11',
 'Causer:6->Result:5b',
 'Causer:3->Result:7',
 'Causer:13->Result:50',
 'Causer:11->Result:14',
 'Causer:50->Result:50']

In [7]:
all_essays = pred_tagged_essays_train + pred_tagged_essays_test
name2essay = {}
for essay in all_essays:
    name2essay[essay.name] = essay

name2crels = essay_to_crels(all_essays, set_cr_tags)
assert len(name2crels) == len(all_essays)

# Train Re-Ranker

## Extract Parses from Sentence Parser

In [7]:
best_top_n = 2
min_feat_freq = 1
best_max_upd = 1 
best_max_parses = 300
best_min_prob = 0.0  # min prob of 0 seems better

In [9]:
def load_rerank(top_n):
    rr_fname = "xs_rerank_" + str(top_n) + ".dill"
    with open(os.path.join(crels_folder, rr_fname), "rb") as f:
        xs_rerank = dill.load(f)

    rr_fname = "xs_rerank_test" + str(top_n) + ".dill"
    with open(os.path.join(crels_folder, rr_fname), "rb") as f:
        xs_test_rerank = dill.load(f)
    return xs_rerank, xs_test_rerank

## Prepare Features

In [10]:
prefixes = [
    "Prob-",
#     "CREL_Pair-",
    "Inv-",
    "num_crels",
    "Tally-",
    "CChain-",
    "CChainStats-",
    "Above-",
    "CREL_",
    "Propn_",
    "Diff_"
]

# Results from run 3
current_best = ['num_crels', 'Inv-', 'Prob-']
best_iterations = 2

# default params (not all of these are optimized)
params = {
    "best_top_n": best_top_n,
#     "best_max_upd": best_max_upd,
    "best_max_parses": best_max_parses,
    "best_min_prob": best_min_prob,
    "min_feat_freq": min_feat_freq
}

## Load Param Hash

In [11]:
vals = ("learning_rate", "best_max_parses", 
        #"best_max_upd", 
        "max_update_items", 
        "best_min_prob", "best_top_n", "extractors", "initial_weight",\
        "min_feat_freq")

def hash_params(params):
    p = dict()
    # only copy over white list vals
    for v in vals:
        p[v] = params[v]
    return str(sorted(p.items())).replace(" ","")

def load_param_hash(db, collection):
    project = {
        "params": "$parameters",
#         "asof":   "$asof",
        "_id": 1
    }
    feats_pipeline = [{ "$project": project }]
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    print("len(rows)", len(rows))
    param_hash = set()
    for r in rows:
        param_hash.add(hash_params(r["params"]))
    return param_hash

In [12]:
import pymongo

client = pymongo.MongoClient(serverSelectionTimeoutMS=100, host="127.0.0.1")
param_hash = load_param_hash(client.metrics_causal_model_reranker, MONGO_COLLECTION)
len(param_hash), MONGO_COLLECTION

len(rows) 0


(0, 'CB_STR_PCPTRN_RE-RANKER_HYPER_PARAM_TD')

In [13]:
MONGO_COLLECTION, best_iterations

('CB_STR_PCPTRN_RE-RANKER_HYPER_PARAM_TD', 2)

## Hyper Parameter Optimization

In [14]:
import datetime
datetime.datetime.now()

datetime.datetime(2019, 6, 24, 18, 8, 36, 955773)

In [15]:
best_f1 = -1
# best_f1 = 0.7457

### Initial Parameters

In [24]:
learning_rate = 0.3
max_update_items = 1  # best_max_upd - 2
initial_weight = 0.01  # was 0.01
best_max_upd = 1

In [17]:
# Form a closure to simplify loop
def trn_mdl(top_n, prms, cv_filtrd):
    
    global best_f1, learning_rate, best_max_upd, initial_weight
    
    n_jobs = None
    # Uses too much RAM, drop to single threaded
    if top_n > 3:        
        n_jobs = 1
    
    f1 = train_model_parallel_logged(
        training_collection_name=MONGO_COLLECTION, results_processor=results_processor,
        feat_extractors=current_best, params=prms,

        cv_folds=cv_filtrd, name2essay=name2essay, 
        learning_rate=learning_rate, max_update_items=best_max_upd, initial_weight=initial_weight,
        set_cr_tags=set_cr_tags,
        # use best iterations from above
        max_epochs=best_iterations, early_stop_iters=best_iterations,
        n_jobs=n_jobs
    )
    if f1 > best_f1:
        best_f1 = f1
        print("New Best F1: {f1:.6f}:\t{params}".format(f1=f1, params=\
                            str((top_n, learning_rate, best_max_upd, initial_weight))))
    else:
        print("         F1: {f1:.6f}:\t{params}".format(f1=f1, params=\
                            str((top_n, learning_rate, best_max_upd, initial_weight))))

In [25]:
param_hash = load_param_hash(client.metrics_causal_model_reranker, MONGO_COLLECTION)

for top_n in [2,3,5,1]:
    
    print("top_n", top_n)
    xs_rr, _ = load_rerank(top_n)
    xs_temp = get_features_from_probabilities(xs_rr, name2crels, best_max_parses, 
                                     causal_model_type=CAUSAL_MODEL_TYPE,
                                     min_feat_freq=min_feat_freq, min_prob=best_min_prob)
    
    cv_flds_rr = cross_validation(xs_temp, 5)
    cv_flds_mm = [min_max_normalize_feats(train, test) for (train, test) in cv_flds_rr]
    
    cv_filtered = []
    for tr, test in cv_flds_mm:
        x_tr,x_test = filter_feats(tr, test, current_best)
        cv_filtered.append((x_tr,x_test))
    
    skipped = False
    for learning_rate in [0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]:
        for initial_weight in [0.01]:                        
            p = {'learning_rate': learning_rate,
                 'best_max_parses': best_max_parses,
                 'best_min_prob': best_min_prob,
                 'best_top_n': top_n,
                 'extractors': list(current_best),
                 'initial_weight': initial_weight,
                 'max_update_items': best_max_upd,
                 'min_feat_freq': min_feat_freq,
            }
            hash_p = hash_params(p)
            if hash_p in param_hash:
                print(".", end = '')
                skipped = True
                continue
            if skipped:
                print()
            trn_mdl(top_n, p, cv_filtered)
            skipped = False

# top_n, best_C, pa_type, best_max_upd, initial_weight

len(rows) 0
top_n 2
         F1: 0.716663:	(2, 0.0005, 1, 0.01)
         F1: 0.732909:	(2, 0.001, 1, 0.01)
         F1: 0.738826:	(2, 0.005, 1, 0.01)
         F1: 0.736862:	(2, 0.01, 1, 0.01)
New Best F1: 0.740970:	(2, 0.05, 1, 0.01)
         F1: 0.740054:	(2, 0.1, 1, 0.01)
         F1: 0.739954:	(2, 0.5, 1, 0.01)
         F1: 0.739248:	(2, 1.0, 1, 0.01)
top_n 3
         F1: 0.719262:	(3, 0.0005, 1, 0.01)
         F1: 0.732728:	(3, 0.001, 1, 0.01)
         F1: 0.739164:	(3, 0.005, 1, 0.01)
         F1: 0.739610:	(3, 0.01, 1, 0.01)
         F1: 0.736781:	(3, 0.05, 1, 0.01)
         F1: 0.738022:	(3, 0.1, 1, 0.01)
         F1: 0.735481:	(3, 0.5, 1, 0.01)
         F1: 0.735237:	(3, 1.0, 1, 0.01)
top_n 5
         F1: 0.722200:	(5, 0.0005, 1, 0.01)
         F1: 0.734831:	(5, 0.001, 1, 0.01)
         F1: 0.739231:	(5, 0.005, 1, 0.01)
         F1: 0.737654:	(5, 0.01, 1, 0.01)
         F1: 0.738402:	(5, 0.05, 1, 0.01)
         F1: 0.738081:	(5, 0.1, 1, 0.01)
         F1: 0.738183:	(5, 0.5, 1, 

# Apply to Test Data

In [26]:
max_update_items, best_max_upd, MONGO_COLLECTION

(1, 1, 'CB_STR_PCPTRN_RE-RANKER_HYPER_PARAM_TD')

In [27]:
min_feat_freq, best_min_prob

(1, 0.0)

### Set Optimal Features

In [29]:
# f1_score	    best_top_n	learning_rate	best_max_parses	max_update_items	initial_weight
# 0.741353    	1	        0.1	            300	           1	                0.01

# # current best settings
# C=best_C, pa_type=pa_type, max_update_items=best_max_upd, initial_weight=initial_weight,

top_n = 1
learning_rate = 0.1
best_max_upd = 1
max_update_items = 1
initial_weight = 0.01

In [30]:
xs_rr, xs_tst_rr = load_rerank(top_n)
xs_temp = get_features_from_probabilities(xs_rr, name2crels, best_max_parses, 
                                 causal_model_type=CAUSAL_MODEL_TYPE,
                                 min_feat_freq=min_feat_freq, min_prob=best_min_prob)

# Don't need CV here, but leave it in anyways
cv_flds_rr = cross_validation(xs_temp, 5)    
xs_test = get_features_from_probabilities(xs_tst_rr, name2crels, best_max_parses, 
                                      causal_model_type=CAUSAL_MODEL_TYPE,
                                      min_feat_freq=min_feat_freq, min_prob=best_min_prob)
xs_train_tmp = []
for train, test in cv_flds_rr:
    xs_train_tmp.extend(test)

xs_train_mm, xs_test_mm = min_max_normalize_feats(xs_train_tmp, xs_test)
# Filter at the end
xs_train_mm_fltr, xs_test_mm_fltr = filter_feats(xs_train_mm, xs_test_mm, current_best) 

### Determine the Optimal Number of Training Iterations

In [31]:
# tuning dataset from training data
num_train = int(0.8 * len(xs_train_mm_fltr))
tmp_train_copy = list(xs_train_mm_fltr)
np.random.shuffle(tmp_train_copy)
tmp_train, tmp_test = tmp_train_copy[:num_train], tmp_train_copy[num_train:]

### Determine Number of Training Iterations

In [32]:
%%time
# use training data to determine number of iterations
mdl = StructuredPerceptron(learning_rate=learning_rate,
                        max_update_items=max_update_items, initial_weight=initial_weight)
# Determine number of training iterations
best_mdl, test_acc_df_ml, best_iterations = train_model(mdl, xs_train=tmp_train, xs_test=tmp_test, name2essay=name2essay, set_cr_tags=set_cr_tags,
     max_epochs=20, early_stop_iters=5, 
     train_instance_fn = train_instance, verbose=True, early_stopping=True)
print("Best iterations:", best_iterations)

Epoch: 0 Train Accuracy: 0.7404 Test Accuracy: 0.7242
Epoch: 1 Train Accuracy: 0.7389 Test Accuracy: 0.7217
Epoch: 2 Train Accuracy: 0.7409 Test Accuracy: 0.7234
Epoch: 3 Train Accuracy: 0.7363 Test Accuracy: 0.7217
Epoch: 4 Train Accuracy: 0.7380 Test Accuracy: 0.7268
Epoch: 5 Train Accuracy: 0.7388 Test Accuracy: 0.7229
Epoch: 6 Train Accuracy: 0.7390 Test Accuracy: 0.7241
Epoch: 7 Train Accuracy: 0.7384 Test Accuracy: 0.7247
Epoch: 8 Train Accuracy: 0.7390 Test Accuracy: 0.7247
Epoch: 9 Train Accuracy: 0.7395 Test Accuracy: 0.7261
Best Test Acc: 0.7268
Best iterations: 5
CPU times: user 58.4 s, sys: 254 ms, total: 58.6 s
Wall time: 58.8 s


In [33]:
test_folds = [(xs_train_mm_fltr, xs_test_mm_fltr)]

params = {
    'learning_rate': learning_rate,
    'best_max_parses': best_max_parses,
    'best_min_prob': best_min_prob,
    'best_top_n': top_n,
    'extractors': list(current_best),
    'initial_weight': initial_weight,
    'max_update_items': best_max_upd,
    'min_feat_freq': min_feat_freq,
}

test_f1 = train_model_parallel_logged(
        training_collection_name=MONGO_TEST_COLLECTION, results_processor=results_processor,
        feat_extractors=current_best, params=params,

        cv_folds=test_folds, 
        
        name2essay=name2essay, 
        learning_rate=learning_rate, max_update_items=best_max_upd, initial_weight=initial_weight,
        set_cr_tags=set_cr_tags,
        # use best iterations from above
        max_epochs=best_iterations, early_stop_iters=best_iterations
    )
test_f1

0.7481617647058824

In [35]:
best_iterations

5